# ICON level 2 data quickview v0.2

In [1]:
!pip install jupyter-dash dash_bootstrap_components --quiet

     |████████████████████████████████| 216 kB 8.2 MB/s 
     |████████████████████████████████| 9.8 MB 44.2 MB/s 
     |████████████████████████████████| 357 kB 47.7 MB/s 


In [2]:
import numpy as np
from datetime import datetime, timedelta,date
import pandas as pd
import os
import glob
from ftplib import FTP_TLS
import fnmatch
import xarray as xr
import re


from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

In [3]:
def download_ICON_spdf_ftps(ftps, target_date_str, ICON_product_long_name,ICON_level='l2'):
    '''
    INPUTS
    ------

    ftps        class : ftps class that has logged in to the ftp site ftp=libftp.ftp('address')
    target_date_str       str : the year of the target day 
    ICON_product_long_name    str : example: l2-2_mighti_vector-wind-green'
    Optional:
    ICON_level       str : 'l1','l2'

    RESULT
    -------
    Download (retrieve) the latest version of ICON data from the FTP site hosting at UC Berkeley (https://icon.ssl.berkeley.edu/Data)

    '''
    target_dir='/pub/data/icon/%s/%s/' % (ICON_level,ICON_product_long_name)
    target_year_dir='%s%s' % (target_dir,target_date_str[0:4])
#     ftps.cwd(target_dir)

#     data_fn='icon*_%s_*.nc' % (target_date_str)#_%s_%s_*.nc' % (ICON_product_long_name,target_date_str)
    if target_year_dir in ftps.nlst(target_dir):
        ftps.cwd(target_year_dir)
        year_dir=ftps.nlst()
        mark_array=np.zeros(np.size(year_dir),dtype=bool)
        for idx,sfn in enumerate(year_dir):
            mark_array[idx]=target_date_str in sfn
        if not np.all(mark_array==False):
            select_fn=np.array(year_dir)[(mark_array)][0]
            download_file = open(select_fn, 'wb')
            ftps.retrbinary('RETR '+ select_fn, download_file.write)
            data_status=True
        else:
            data_status=False
    else:
        data_status=False
    return data_status

def get_ICON_data_spdf(tar_date,ICON_product_long_name,ICON_level='l2'):

    # '''Insert a target day'''
    target_datetime=datetime.strptime(tar_date, '%Y-%m-%d')
    # (year, month, day)
    target_YYYY='%4i' % target_datetime.year
    target_mm='%02i' % target_datetime.month
    target_dd='%02i' % target_datetime.day
    target_doy='%03i' % ((target_datetime-datetime(np.int(target_YYYY),1,1)).days+1)
    target_date_str='%s%s%s' % (target_YYYY,target_mm,target_dd)
    target_dir='/pub/data/icon/%s/%s/%s/' % (ICON_level,ICON_product_long_name,target_date_str[0:4])
    data_fn='icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str)
    data_status=True
    if not glob.glob(data_fn):
    # print('yep')

        ftps=FTP_TLS('spdf.gsfc.nasa.gov')

        ftps.login()
        data_status=download_ICON_spdf_ftps(ftps, target_date_str, ICON_product_long_name)
        ftps.quit()
        if data_status:
            select_fn=glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]
            dn_fn='https://spdf.gsfc.nasa.gov'+target_dir+select_fn
            return  'Download: %s %s' % ( target_date_str,ICON_product_long_name),dn_fn,' '

        else:
            return ' ','https://spdf.gsfc.nasa.gov','%s is not available!' % data_fn
    else:
        select_fn=glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]
        dn_fn='https://spdf.gsfc.nasa.gov'+target_dir+select_fn
#     if not data_status:
#         return dash.no_update, '%s not exsits!' % select_fn
        return  'Download: %s %s' % ( target_date_str,ICON_product_long_name),dn_fn,' '


def get_ICON_variable_spdf(tar_date,ICON_product_long_name,select_var):
  # '''Insert a target day'''
    target_datetime=datetime.strptime(tar_date, '%Y-%m-%d')
    # (year, month, day)
    target_YYYY='%4i' % target_datetime.year
    target_mm='%02i' % target_datetime.month
    target_dd='%02i' % target_datetime.day
    target_doy='%03i' % ((target_datetime-datetime(np.int(target_YYYY),1,1)).days+1)
    target_date_str='%s%s%s' % (target_YYYY,target_mm,target_dd)

    ICON_product=ICON_product_long_name.split('_')[0]
    if (ICON_product == 'l2-2') :
        '''MIGHTI L2.2 Vector Wind'''
        fn_L22=glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]
        # fn_L22='ICON_L2-2_MIGHTI_Vector-Wind-Red_2020-10-10_v04r000.NC'
        # print(re.split('_|-',fn_L22))
        dm_g=xr.open_dataset(fn_L22)
        test=fn_L22
        dm_g=dm_g.rename({'Epoch': 'time_ms',
                        'ICON_L22_Orbit_Number':'orb_num',
#                         'ICON_L22_Longitude':'tang_lon',
#                         'ICON_L22_Latitude':'tang_lat',
                        'ICON_L22_Altitude':'tang_alt',
                        'ICON_L22_Meridional_Wind': 'umer',
                        'ICON_L22_Zonal_Wind':'uzon',
                        'ICON_L22_Local_Solar_Time':'tang_slt',
                        'ICON_L22_Wind_Quality':'wind_quality'})

        dm_g['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dm_g['time_ms'].values]))
        dm_g['tang_lon']=dm_g.ICON_L22_Longitude.transpose()
        dm_g['tang_lat']=dm_g.ICON_L22_Latitude.transpose()
        # Only allow the good data: dm_g.wind_quality >= 0.5
        good_data=(dm_g.wind_quality == 0.5)|(dm_g.wind_quality == 1)
        dm_g[['umer','uzon']]=dm_g[['umer','uzon']].where(good_data)
        tar_xr=dm_g
        if (select_var == 'Meridional Wind (m/s)'):
            sel_var=tar_xr.umer.transpose()
          # stop
        if (select_var == 'Zonal Wind (m/s)'):
            sel_var=tar_xr.uzon.transpose()
    
    if (ICON_product == 'l2-3'):
        '''MIGHTI L2.3 Temperature'''


        fn_L23=glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]
        AB=(re.split('_|-',fn_L23)[4]).upper()

        dTN=xr.open_dataset(fn_L23)
        dTN=dTN.rename({'Epoch': 'time_ms',
                        'ICON_L23_Orbit_Number':'orb_num',
                        'ICON_L23_MIGHTI_%s_Tangent_Longitude' % (AB):'tang_lon_orig',
                        'ICON_L23_MIGHTI_%s_Tangent_Latitude' % (AB):'tang_lat_orig',
                        'ICON_L23_MIGHTI_%s_Tangent_Altitude' % (AB):'tang_alt',
                        'ICON_L23_MIGHTI_%s_Temperature' % (AB): 'TN',
                        'ICON_L23_MIGHTI_%s_Tangent_Local_Solar_Time' % (AB):'tang_slt',
                        'ICON_L1_MIGHTI_%s_Quality_Flag_South_Atlantic_Anomaly' % (AB):'saa_flag',
                        'ICON_L1_MIGHTI_%s_Quality_Flag_Bad_Calibration' % (AB):'cali_flag'
                        })
        dTN['tang_lon']=dTN.tang_lon_orig.transpose()
        dTN['tang_lat']=dTN.tang_lat_orig.transpose()
        good_data = (dTN.saa_flag == 0) & (dTN.cali_flag == 0) & (dTN.TN > 0) & (abs(dTN.TN) < 10000)
        dTN['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dTN['time_ms'].values]))
        dTN['TN']=dTN['TN'].where(good_data)
        tar_xr=dTN
        # stop
        if (select_var == 'Neutral Temperature (K)'):
            sel_var=tar_xr.TN

    if (ICON_product == 'l2-4') :

        fn_L24 =glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]

        dON2=xr.open_dataset(fn_L24)
        dON2=dON2.rename({'Epoch': 'time_ms',
                        'ICON_L24_Disk_ON2': 'disk_ON2',
                        'ICON_L24_Local_Solar_Time_Disk': 'disk_slt',
                        'ICON_L24_Disk_Longitude': 'tang_lon',
                        'ICON_L24_Disk_Latitude': 'tang_lat',
                        'ICON_L24_Disk_Magnetic_Longitude': 'disk_mlon',
                        'ICON_L24_Disk_Magnetic_Latitude': 'disk_mlat',
                        'ICON_L24_Disk_Retrieval_Flags': 'model_flags'
                        })

        good_data = (dON2.model_flags == 0)
        dON2['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dON2['time_ms'].values]))
        dON2['disk_ON2']=dON2['disk_ON2'].where(good_data)
        tar_xr=dON2
        if (select_var == 'Disk O/N2'):
            sel_var=tar_xr.disk_ON2

    if (ICON_product == 'l2-5'):
        '''FUV L2.5 Nighttime Ionosphere'''
        fn_L25 =glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]

        # Load L2.5 Nighttime plasma observations #
        dL25=xr.open_dataset(fn_L25)
        dL25=dL25.rename({'Epoch': 'time_ms',
                          'ICON_L25_HMF2': 'HmF2',
                          'ICON_L25_NMF2': 'NmF2',
                          'ICON_L25_O_Plus_Density': 'O_plus',
                          'ICON_L25_O_Plus_Profile_Altitude': 'tang_alt',
                          'ICON_L25_Orbit_Number': 'orb_num',
                          'ICON_L25_Local_Solar_Time': 'tang_slt',
                          'ICON_L25_Longitude': 'tang_lon',
                          'ICON_L25_Latitude': 'tang_lat',
                          'ICON_L25_Magnetic_Longitude': 'HmF2_mlon',
                          'ICON_L25_Magnetic_Latitude': 'HmF2_mlat',
                          'ICON_L25_Quality': 'L25_flag'
                          })

        dL25['time']  = pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dL25['time_ms'].values]))
        good_data = (dL25.L25_flag== 1)# 
        dL25[['HmF2','NmF2']]=dL25[['HmF2','NmF2']].where(good_data)
        tar_xr=dL25
        # 'O+ Density (/cc)','HmF2 (km)', 'NmF2 (/cc)'
        if (select_var == 'HmF2 (km)'):
            sel_var=tar_xr.HmF2[:,2]
        if (select_var == 'NmF2 (/cc)'):
            sel_var=tar_xr.NmF2[:,2]
        if (select_var == 'O+ Density (/cc)'):
            sel_var=tar_xr.O_plus[:,:,2]

    if (ICON_product == 'l2-6'):
        '''EUV L2.6 Daytime Ionosphere'''
        fn_L26 =glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]

        # Load L2.6 Daytime plasma observations #
        dEUV =xr.open_dataset(fn_L26)

        dEUV=dEUV.rename({'Epoch': 'time_ms',
                          'ICON_L26_HmF2': 'HmF2',
                          'ICON_L26_NmF2': 'NmF2',
                          'ICON_L26_Oplus': 'O_plus',
                          'ICON_L26_Orbit_Number': 'orb_num',
                          'ICON_L26_Local_Solar_Time': 'tang_slt',
                          'ICON_L26_Longitude': 'tang_lon',
                          'ICON_L26_Latitude': 'tang_lat',
                          'ICON_L26_Altitude': 'tang_alt',
                          'ICON_L26_Magnetic_Longitude': 'tang_mlon',
                          'ICON_L26_Magnetic_Latitude': 'tang_mlat',
                          'ICON_L26_Flag': 'L26_flag'
                          })

        dEUV['time']  = pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dEUV['time_ms'].values]))
        good_data = (dEUV.L26_flag == 1) | (dEUV.L26_flag == 0)# 
        # dEUV=dEUV.where(good_data)
        dEUV[['NmF2','HmF2','O_plus']]=dEUV[['NmF2','HmF2','O_plus']].where(good_data)
        tar_xr=dEUV
        # 'O+ Density (/cc)','HmF2 (km)', 'NmF2 (/cc)'
        if (select_var == 'HmF2 (km)'):
            sel_var=tar_xr.HmF2
        if (select_var == 'NmF2 (/cc)'):
            sel_var=tar_xr.NmF2

    if (ICON_product == 'l2-7'):
        '''IVM L2.7 In-situ Plasma Measurement'''
        fn_L27 =glob.glob('icon_%s*%s_*.nc' % (ICON_product_long_name,target_date_str))[0]
        print(fn_L27)
        # #### Load ion velocity
        di =xr.open_dataset(fn_L27)

        di=di.rename({'Epoch': 'time_ms',
                          'ICON_L27_Ion_Velocity_Meridional': 'vmer',
                          'ICON_L27_Ion_Velocity_Zonal': 'vzon',
                          'ICON_L27_Ion_Density': 'Ni',
                          'ICON_L27_Ion_Temperature': 'Ti',
                          'ICON_L27_Fractional_Ion_Density_O': 'Oplus_frac',
                          'ICON_L27_Orbit_Number': 'orb_num',
                          'ICON_L27_Solar_Local_Time': 'slt',
                          'ICON_L27_Longitude': 'tang_lon',
                          'ICON_L27_Latitude': 'tang_lat',
                          'ICON_L27_Altitude': 'tang_alt',
                          'ICON_L27_Magnetic_Longitude': 'mlon',
                          'ICON_L27_Magnetic_Latitude': 'mlat',
                          'ICON_L27_DM_Flag': 'DM_flag',
                          'ICON_L27_RPA_Flag': 'RPA_flag'
                          })

        di['time']  = pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in di['time_ms'].values]))
        good_drift_data = (di.DM_flag == 0) & (di.RPA_flag < 2)# 
        good_data= (di.RPA_flag < 2)
        di[['Ti','Ni','Oplus_frac']]=di[['Ti','Ni','Oplus_frac']].where(good_data)
        di[['vmer','vzon']]=di[['vmer','vzon']].where(good_drift_data)
        tar_xr=di

        if (select_var =='Meridional Drift (m/s)'):
            sel_var=tar_xr.vmer
        if (select_var =='Zonal Drift (m/s)'):
            sel_var=tar_xr.vzon
        if (select_var =='Ion Density (/cc)'):
            sel_var=tar_xr.Ni
        if (select_var =='Ion Temperature (K)'):
            sel_var=tar_xr.Ti
        if (select_var =='O+ Fraction'):
            sel_var=tar_xr.Oplus_frac


    return sel_var, tar_xr


In [4]:
# app = Dash()
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.UNITED])
fnameDict = {'l2-2_mighti_vector-wind-green': ['Meridional Wind (m/s)', 'Zonal Wind (m/s)'],
             'l2-2_mighti_vector-wind-red': ['Meridional Wind (m/s)', 'Zonal Wind (m/s)'], 
             'l2-3_mighti-a_temperature': ['Neutral Temperature (K)'],
             'l2-3_mighti-b_temperature': ['Neutral Temperature (K)'],
             'l2-4_fuv_day': ['Disk O/N2'],
             'l2-5_fuv_night': ['HmF2 (km)', 'NmF2 (/cc)'],
             'l2-6_euv': ['HmF2 (km)', 'NmF2 (/cc)'],
             'l2-7_ivm-a': ['Meridional Drift (m/s)','Zonal Drift (m/s)',
                          'Ion Density (/cc)','O+ Fraction', 'Ion Temperature (K)']}



names = list(fnameDict.keys())

app.layout = html.Div([
#     dbc.Spinner(spinner_style={"width": "3rem", "height": "3rem"}),
    html.H1('Ionospheric Connection Explorer Level 2 Data Quickview v0.2'),           
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=date(2019, 12, 6),
        max_date_allowed=date.today(),
        initial_visible_month=date(2020, 10, 10),
        date=date(2020, 10, 10)
    ),
    html.Div(id='output-container-date-picker-single'),

    dcc.Dropdown( 
        id='dataFull_dropdown',
        options=[{'label':name, 'value':name} for name in names]
            ),
    
    
    dcc.Loading(children=[html.Div(id='test-output-container')],color='#119DFF',type='dot'),

    html.Div(id='error-output-container',style={'color': 'red'}),
    dcc.Dropdown(
            id='opt_dropdown'           
            ),


    html.Div(id='display-selected-values'),
    
#     dcc.Link('Go to data', href= 'test/test'),
#     html.Div(id='test2-output-container')
    dcc.Loading(children=[dcc.Graph(id='graph')],color='#119DFF',type='dot'),

    html.Div([], id='download-url-container'),

    html.Div('Contact: yjwu at ssl.berkeley.edu')
   ])


In [5]:
'''============ Select a day ================='''
@app.callback(
    Output('output-container-date-picker-single', 'children'),
    Input('my-date-picker-single', 'date'))
def update_output(date_value):
    string_prefix = 'You have selected: '
    if date_value is not None:
        date_object = date.fromisoformat(date_value)
        date_string = date_object.strftime('%B %d, %Y')
        return string_prefix + date_string


'''========== Set the default variable of the data product dropdown ======='''
@app.callback(
    Output('dataFull_dropdown', 'value'),
    [Input('my-date-picker-single', 'date'),
     Input('dataFull_dropdown', 'options')
     ])
def set_DP_value(date_value,available_options):
     return available_options[0]['value']

'''============ Download the data ============='''
@app.callback(
    [Output('test-output-container', 'children'),
     Output('download-url-container', 'children'),
     Output('error-output-container', 'children')
    ],
    [
    Input('my-date-picker-single', 'date'),
    Input('dataFull_dropdown', 'value')
     ]
)

def update_output_download(date_value,pd_value):

    test_o3,data_url,error_msg=get_ICON_data_spdf(date_value, pd_value)
    # print('done!')
    return test_o3, [html.A('Download data', href=data_url, target="_blank")], error_msg
'''============ Select a data product ========='''
@app.callback(
    Output('opt_dropdown', 'options')
,
    [Input('dataFull_dropdown', 'value'),
     Input('test-output-container', 'children')
     ]
)
def update_date_dropdown(name,makesure_download):
    return [{'label': i, 'value': i} for i in fnameDict[name]]

'''========== Set the default variable of the variable dropdown ======='''
@app.callback(
    Output('opt_dropdown', 'value'),
    [Input('opt_dropdown', 'options'),
     Input('test-output-container', 'children')
    ]
    )
def set_variable_value(available_var_options,makesure_download):
    return available_var_options[0]['value']

'''============ Select a variable ============'''
@app.callback(
   Output('display-selected-values', 'children'),
    [Input('opt_dropdown', 'value'),
     Input('test-output-container', 'children')
    ])
def set_display_children(selected_value,makesure_download):
    return 'You have selected {}'.format(selected_value)

'''============ plot the variable ============='''
@app.callback(
    Output('graph', 'figure'),
    [
    Input('my-date-picker-single', 'date'),
    Input('dataFull_dropdown', 'value'),
    Input('opt_dropdown', 'value'),
    Input('test-output-container', 'children'),
    Input('error-output-container', 'children')]
)

def update_figure_daily(date_value,pd_value,selected_value,makesure_download,error_msg):
    if (error_msg == ' '):

        sel_var, tar_xr=get_ICON_variable_spdf(date_value, pd_value,selected_value)
        # print('done!')
        if (np.size(sel_var.dims) == 1):
            fig=px.line(
                x=tar_xr.time, y=sel_var, # replace with your own data source
                title=pd_value, markers=True,
                labels={'x':'UTC Time','y':selected_value})
            fig.update_traces(customdata=np.stack((tar_xr['tang_lon'],tar_xr['tang_lat']),axis=-1),
                   hovertemplate =
                   "%{x}<br>"+\
                   "y: %{y}<br>"+\
                   "lon,lat: %{customdata[0]:6.2f}, %{customdata[1]:6.2f}")
            return fig

        if (np.size(sel_var.dims) == 2):
            if (selected_value in ['Meridional Wind (m/s)', 'Zonal Wind (m/s)']):
                zmax,zmin=250,-250
                colorscale='RdBu'
                x=tar_xr.time # horizontal axis
                y=tar_xr.tang_alt # vertical axis
                z=sel_var
            if (selected_value in ['Neutral Temperature (K)']):
                zmax,zmin=500,100
                colorscale='plasma'
                x=tar_xr.time
                y=np.nanmean(tar_xr.tang_alt.values,axis=0) # vertical axis
                z=sel_var.transpose()


            layout=go.Layout(
                title=selected_value,
                xaxis=dict(title='UTC Time'),
                yaxis=dict(title='Altitude (km)'),
                plot_bgcolor='#A9A9A9'
                  )
            data=go.Heatmap(

                x=x, # horizontal axis
                y=y, # vertical axis
                z=z,
                colorscale=colorscale,
                zmax=zmax, zmin=zmin)
            
            fig=go.Figure(data=data,layout=layout)
            fig.update_traces(customdata=np.stack((tar_xr['tang_lon'],tar_xr['tang_lat']),axis=-1),
                   hovertemplate =
                   "%{x}<br>"+\
                   "y: %{y}<br>"+\
                   "z: %{z}<br>"+\
                   "lon,lat: %{customdata[0]:6.2f},  %{customdata[1]:6.2f}<extra></extra>")
            return fig
    else:
        fig=px.line(
        x=[0,1], y=[0,1], # replace with your own data source
        title='NOT AVAILABLE', markers=True,
        labels={'x':'UTC Time','y':'NO DATA'})
        return fig


In [6]:
app.run_server(mode = 'external', port=8050)

Dash app running on:


<IPython.core.display.Javascript object>